In [1]:
import time
import pandas as pd
import numpy as np
import math
from statistics import mean
import statistics
import itertools
from scipy.stats import norm
import warnings

In [2]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
# single item 9 non-identical stores expected normal demands (mean, st.dev).
daily_demands = {'1': [(42.1,7.6),(41.9,12),(40.2,12.3),(37.2,12.3),(36.1,12.3),(34.6,12.6),(34.3,12.7)],
                '2': [(15.6,4.8),(15.3,7.9),(15.8,7.7),(14.8,8.1),(16.2,7.7),(15.9,7.8),(17.2,7.7)],
                '3': [(30,8.2),(29.3,12.3),(27.2,12.9),(26,14),(25.9,14.1),(25.6,14.4),(26.1,14.3)],
                '4': [(3.1,1.6),(2.8,2.5),(3,2.5),(2.5,2.6),(2.8,2.6),(2.7,2.6),(3.2,2.5)],
                '5': [(7.9,3.1),(8.4,4.8),(7.9,4.8),(7.3,5),(7.4,5),(6.9,5),(8.1,5)],
                '6': [(22.8,5.8),(21.4,9),(19.9,9.1),(16.4,10.1),(20.4,10.2),(20.1,10.4),(16.6,10.4)],
                '7': [(3.1,1.5),(2.9,2.5),(3.6,2.5),(3.1,2.4),(5,2.4),(5,2.5),(4.1,2.4)],
                '8': [(3.8,2.4),(4.2,3.8),(4,3.7),(4,4.1),(3.4,4),(3.4,3.9),(3.4,3.8)],
                '9': [(7.9,3.9),(8.7,5.6),(9,5.6),(9.1,6),(9.4,6),(9.4,6),(9.3,5.9)]}

In [4]:
store_grades = {'1': 4, '2': 3, '3': 4, '4': 2, '5': 2, '6': 3, '7': 1, '8': 1, '9': 3}

In [5]:
current_inventory={}
total=0
for store, demand in daily_demands.items():
    total =total+(sum(x[0] for x in demand))


weight=0
for k in store_grades.values():
    weight=weight+k
    
for i in daily_demands.keys():
    current_inventory[i]=math.ceil((int(total)*(store_grades[i]/weight)))

In [6]:
initial_inventory = current_inventory.copy()

In [7]:
def reset_inventory():
    return initial_inventory.copy()

In [8]:
beginning_inventory=current_inventory.copy()

In [9]:
costs = {'stockout': 30,
        'holding': 1,
        'transshipment': 1}

In [10]:
num_stores = 9
current_day = 0

In [11]:
def convolution(store_id, start_day):
    
    store_data = daily_demands[store_id]
    future_data = store_data[start_day:]
    mean_values = [mean for mean, _ in future_data]
    std_dev_values = [std_dev for _, std_dev in future_data]
    convoluted_mean = sum(mean_values)
    convoluted_st_dev = math.sqrt(sum(x**2 for x in std_dev_values))
    return convoluted_mean, convoluted_st_dev

In [12]:
p_value = (costs['stockout'] - costs['transshipment']) / (costs['stockout'] + costs['holding'] - costs['transshipment'])
ratio=norm.ppf(p_value)

In [13]:
current_day = 0
sales_dff = pd.DataFrame(columns=['Store', 'Day', 'Sales','Lost Sales','Demand','Inventory'])

In [14]:
def update_inventory(i,current_day):
    global sales_dff
    todays_sales = {i: max(0,int(np.random.normal(daily_demands[(i)][current_day][0], daily_demands[(i)][current_day][1])))
                    }
    beforesales_inv=current_inventory[i]
    actual_sales = min(todays_sales[i], current_inventory[i])
    lost_sales = max(0, todays_sales[i] - current_inventory[i])
    current_inventory[i] = current_inventory[i]-actual_sales
    new_row = pd.DataFrame({'Store': [i],"Week":[week], 'Day': [current_day], 'Sales': [actual_sales],'Lost Sales':lost_sales,'Demand':todays_sales[i],'Inventory':current_inventory[i],'Inv bef sales': beforesales_inv})
   
    sales_dff = pd.concat([sales_dff, new_row], ignore_index=True)
    
    return current_inventory   

In [15]:
def reset_inventory():
    return initial_inventory.copy()

In [16]:
def shipper_candidates(current_day):
    shipper_cand = []
   
    
    for i in current_inventory.keys():
        mean, st_dev = convolution(i, current_day)
        critical_point = mean + (ratio*st_dev)
        
        if critical_point < update_inventory(str(i),current_day)[i]:
            shipper_cand.append(i)
    return shipper_cand

In [17]:
def receiving_candidates(current_day):
    receiving_cand = []
    for i in current_inventory.keys():
        mean, st_dev = convolution(i, current_day)
        critical_point = mean - (ratio*st_dev)
      
        if critical_point > current_inventory[i]:
            receiving_cand.append(i)
    return receiving_cand


In [18]:
def converge_parwise(cost_list, threshold):
    pairwise_delta = [abs(a - b) for a, b in itertools.combinations(cost_list, 2)]
    max_delta = max(pairwise_delta)
    
    return max_delta < threshold

In [19]:
current_time=time.time()
max_week=2000
week = 1
sim_check=100
threshold=5
results = pd.DataFrame()
resultsale = pd.DataFrame()
cost_week=[]
expected_costs = []
while (week<=max_week):
    for _ in range(sim_check):
    
        print(f"week ------------------- {week}")
        current_inventory = reset_inventory()
        #print(f"{week}. beginning of the week, inventory: {current_inventory}")
        weekly_df = pd.DataFrame(columns=["week", "day", "shipper", "receiver", "transshipment_amount"])
        sales_dff = pd.DataFrame()
        current_day = 0
        for i in range(6):
            if current_day <= 5:
                empty_list=[]
                shipping_list = shipper_candidates(current_day)
                receiving_list = receiving_candidates(current_day)
                daily_df = pd.DataFrame(columns=["shipper", "receiver", "transshipment_amount", "transshipment_cost"])
                for i in shipping_list:
                    mean,stdev=convolution(i,current_day)            
                    
                    for k in receiving_list:
                        mean2,stdev2=convolution(k,current_day)
                        best_x = 0
                        min_cost = float('inf')
                        s_inv=current_inventory[i]
                        r_inv=current_inventory[k]
                        #print("shipper inventory",s_inv,"mağaza no:",i)
                        #print("receiver inventory",r_inv,"mağaza no",k)
                        x_lower=0
                        quantity_max=max(0,beginning_inventory[k]-r_inv)
                        x_upper=min(s_inv,quantity_max)
                        
                        tolerance=1
                        epsilon=1
                        min_cost=float("inf")
                        best_x=None

                        while (x_upper-x_lower>tolerance):
                            mid=(x_upper+x_lower)//2
                            mid_left=max(mid-epsilon,x_lower)
                            cost_eps =0
                            mid_eps = min(mid + epsilon, x_upper)
                            sum_cost=0
                            new_list=[]
                            hold_cv={}
                            
                            for number in range(100):
                                (exp_shipping)=int(np.random.normal(mean,stdev))
                                (exp_receiver)=int(np.random.normal(mean2,stdev2))

                                cost = (costs['stockout'] * max(0, exp_receiver - (current_inventory[k] + mid_left))) + \
                               (costs['stockout'] * max(0, exp_shipping - (current_inventory[i] - mid_left))) + \
                               (costs['transshipment'] * mid_left)
                                sum_cost+=cost
                                new_list.append(cost)

                            expected_cost=sum_cost/100
                            st_dev=np.std(new_list)

                            CV=(st_dev/expected_cost)*100
                            hold_cv[mid]=CV
                            
                            cost_eps =0
                            mid_eps = min(mid + epsilon, x_upper)
                            for number in range(100):
                                exp_shipping=np.random.normal(mean,stdev)
                                exp_receiver=np.random.normal(mean2,stdev2)

                                cost2 = (costs['stockout'] * max(0, exp_receiver - (current_inventory[k] + mid_eps))) + \
                               (costs['stockout'] * max(0, exp_shipping - (current_inventory[i] - mid_eps))) + \
                               (costs['transshipment'] * mid_eps)
                                cost_eps+=cost2
                            expected_costeps=cost_eps/100
                            if (expected_cost<expected_costeps):
                                x_upper=mid
                                if (expected_cost<min_cost):
                                    min_cost=expected_cost
                                    best_x=mid_left
                            else:
                                x_lower=mid
                                if (expected_costeps<min_cost):
                                    min_cost=expected_costeps
                                    best_x=mid_eps

                        daily_df = pd.concat([daily_df, pd.DataFrame([{
                        "shipper": i,
                        "receiver": k,
                        "transshipment_amount": best_x,
                        "transshipment_cost": min_cost
                }])], ignore_index=True)

                while not daily_df.empty:

                    min_idx = daily_df["transshipment_cost"].idxmin()
                    min_cost_row = daily_df.loc[min_idx]
                    deleted_shipper = min_cost_row["shipper"]
                    deleted_receiver = min_cost_row["receiver"]
                    transhipment_amount = daily_df.loc[min_idx, "transshipment_amount"]

                    current_inventory[str(deleted_shipper)] = current_inventory[str(deleted_shipper)] - int(transhipment_amount)
                    current_inventory[str(deleted_receiver)] = current_inventory[str(deleted_receiver)] + int(transhipment_amount)

                    weekly_df = pd.concat([weekly_df, pd.DataFrame([{
                        "week": week,
                        "day": current_day,
                        "shipper": deleted_shipper,
                        "receiver": deleted_receiver,
                        "transshipment_amount": transhipment_amount

                }])], ignore_index=True)

                    daily_df = daily_df[(daily_df["shipper"] != deleted_shipper) & (daily_df["receiver"] != deleted_receiver)]

                #print(f"finished day: {current_day}, inventory_levels: {current_inventory}") 
                #print("-------------------------------")
                current_day = current_day + 1
        if current_day == 6:
            for i in current_inventory.keys():

                mean,stdev=daily_demands[i][current_day]
                number=int(np.random.normal(mean,stdev))
                beforesales_inv=current_inventory[i]
                actual_sales = min(number, current_inventory[i])
                lost_sales = max(0, number - current_inventory[i])
                current_inventory[i]=current_inventory[i]-actual_sales
                new_row = pd.DataFrame({'Store': [i],"Week":[week], 'Day': [current_day], 'Sales': actual_sales,'Lost Sales':lost_sales,'Demand':number,'Inventory':current_inventory[i],'Inv bef sales': beforesales_inv})
                sales_dff = pd.concat([sales_dff, new_row], ignore_index=True)
        #print(sales_dff)
        weekly_df["transshipment_amount"] = pd.to_numeric(weekly_df["transshipment_amount"], errors="coerce")
        sales_dff["Inventory"] = pd.to_numeric(sales_dff["Inventory"], errors="coerce")
        sales_dff["Lost Sales"]=pd.to_numeric(sales_dff["Lost Sales"], errors="coerce")
        weekly_cost = int(costs["transshipment"])*weekly_df["transshipment_amount"].sum()+int(costs["stockout"])*sales_dff["Lost Sales"].sum()+int(costs["holding"])*sales_dff["Inventory"].sum()
        weekly_df["week_cost"] = weekly_cost
        #print(weekly_df)
        results = pd.concat([results, weekly_df], ignore_index=True)
        resultsale=pd.concat([resultsale, sales_dff], ignore_index=True)
        cost_week.append(weekly_cost)
        week = week + 1
        
    current_expected = np.mean(cost_week)
    expected_costs.append(current_expected)
    print(f"in iteration {week-1} the expected cost: {current_expected:.2f}")
   
    if len(expected_costs) >= 2:
        delta = abs(expected_costs[-1] - expected_costs[-2])
        print("Delta:",delta)
        if delta<=threshold:
            print(f" Convergence is occured. Iteration stopped in week {week}")
            break
        last_iteration = week - 1

print(resultsale)
print(expected_costs)            
print("Passing time to run the program: ",time.time()-current_time)   

week ------------------- 1
week ------------------- 2
week ------------------- 3
week ------------------- 4
week ------------------- 5
week ------------------- 6
week ------------------- 7
week ------------------- 8
week ------------------- 9
week ------------------- 10
week ------------------- 11
week ------------------- 12
week ------------------- 13
week ------------------- 14
week ------------------- 15
week ------------------- 16
week ------------------- 17
week ------------------- 18
week ------------------- 19
week ------------------- 20
week ------------------- 21
week ------------------- 22
week ------------------- 23
week ------------------- 24
week ------------------- 25
week ------------------- 26
week ------------------- 27
week ------------------- 28
week ------------------- 29
week ------------------- 30
week ------------------- 31
week ------------------- 32
week ------------------- 33
week ------------------- 34
week ------------------- 35
week ------------------- 36
w

In [20]:
results.head()

,week,day,shipper,receiver,transshipment_amount,week_cost
0,1,0,9,6,31,4610
1,1,0,4,3,37,4610
2,1,1,4,1,23,4610
3,1,4,5,4,11,4610
4,1,5,9,3,21,4610


In [21]:
results["transshipment_amount"].sum()/last_iteration

201.78

In [22]:
resultsale["Inventory"].sum()/last_iteration

3733.173333333333

In [23]:
resultsale["Lost Sales"].sum()/last_iteration

69.24

In [24]:
unique_week_costs = results.drop_duplicates(subset="week")[["week_cost"]]

mean_cost = unique_week_costs["week_cost"].mean()
deviation_cost = unique_week_costs["week_cost"].std()

# coefficient of variation
CV=(deviation_cost/mean_cost)*100
print(CV)
print(mean_cost)

21.91406330475392
4509.115


In [25]:
resultsale.head(63)

,Store,Week,Day,Sales,Lost Sales,Demand,Inventory,Inv bef sales
0,1,1,0,36,0,36,120,156
1,2,1,0,22,0,22,95,117
2,3,1,0,38,0,38,118,156
3,4,1,0,4,0,4,74,78
4,5,1,0,4,0,4,74,78
...,...,...,...,...,...,...,...,...
58,5,1,6,7,0,7,17,24
59,6,1,6,19,0,19,6,25
60,7,1,6,0,0,0,7,7
61,8,1,6,10,0,10,0,10
